# TPC3 - Modelo de Ising de dimensão infinita com anisotropia

*  Não deve escrever o seu nome nem nenhum código identificador em nenhum
sítio. 
* Deve entregar ESTE Notebook de Jupyter.
* O trabalho é individual. Podem e devem haver discussões com colegas
mas o resultado entregue é individual. 
* Não deve acrescentar ou apagar nenhuma célula.
* Todas as explicações devem ser claras e concisas. 
* É preferível fazer menos e fazer bem que muito e mal. 
* A não verificação de alguma destas regras leva ao anulamento e exclusão
da prova.

In [2]:
# Importe os Módulos Necessários
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

## Pergunta 1
Nas aulas discutimos a solução do modelo de Ising de campo médio com
$N$ spins ($s=\pm1/2$). Neste trabalho de casa vamos abordar o modelo
de Ising de campo médio de spin um ($s_{i}=\pm1,0$) com anisotropia.
Considere o Hamiltoniano, 
\begin{equation}
H(M,N_0,N)=\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^{N}\left(1-s_{i}s_{j}\right)+D\sum_{i=1}^{N}s_{i}^{2}=\frac{N^{2}-M^{2}}{N}+D(N-N_{0})
\end{equation}
onde a magnetização é dada por $M=\sum_{i=1}^{N}s_{i}$ e as variáveis
de spin $s_{i}$ são $\{\pm1\}$. Para este modelo a energia é função
apenas da magnetização e do número de partículas. A densidade de estados
do sistema é obtida usando o número de spins $s_{i}=+1$ ($N_{+}$),
o número de spins $s_{i}=-1$ ($N_{-}$) e o número de spins $s_{i}=0$
($N_{0}$): 
\begin{align*}
\Omega(N_{+},N_{-},N_{0}) & =\frac{N!}{N_{+}!N_{-}!N_{0}!} & N_{+} & =\frac{N-N_{0}+M}{2} & N_{-} & =\frac{N-N_{0}-M}{2}
\end{align*}
Com esta parametrização,
\begin{equation}
\Omega(M,N_{0},N)=\frac{N!}{\left(\frac{N-N_{0}+M}{2}\right)!\left(\frac{N-N_{0}-M}{2}\right)!N_{0}!}
\end{equation}
onde $M\in\{-(N-N_{0}),-(N-N_{0})+2,\dots,(N-N_{0})-2,(N-N_{0})\}$,
$N_{0}\in\{0,\dots,N\}$.

1. Calcule numericamente o valor médio exacto como função da temperatura
do módulo da magnetização para $\beta$ entre 0.1 e 1.5 e para valores
de $D\in\{0.0,0.25,0.5,0.75,1\}$
\begin{align*}
\left\langle \frac{|M|}{N}\right\rangle  & =\sum_{M,N_{0}}\frac{|M|}{N}\Omega(M,N_{0},N)\frac{e^{-\beta H(M,N_{0},N)}}{Z(\beta)}\\
Z(\beta) & =\sum_{M,N_{0}}\Omega(M,N_{0},N)e^{-\beta H(M,N_{0},N)}
\end{align*}
Sugestão: O cálculo da combinatórica envolve números muito grandes.
O idela é usar a fórmula de Stirling,
\begin{equation}
\log(n!)=n\log(n)-n+\frac{1}{2}\log(2\pi n)
\end{equation}
e notar que o valor médio fica inalterado se deslocar a energia $-\beta H(M,N_{0},N)\to-\beta H(M,N_{0},N)-C$
nas exponenciais. 

## Alínea 1

Vamos verificar que o valor médio fica inalterado pela translação da energia sugerida:
$$
\begin{align*}
\left\langle \frac{|M|}{N}\right\rangle  &= \sum_{M,N_{0}}\frac{|M|}{N}\Omega(M,N_{0},N)\frac{e^{-\beta H(M,N_{0},N)}}{\sum_{M,N_{0}}\Omega(M,N_{0},N)e^{-\beta H(M,N_{0},N)}}
\\[0.2cm]
&= \sum_{M,N_{0}}\frac{|M|}{N}\Omega(M,N_{0},N)\frac{e^{-\beta H(M,N_{0},N)-C}}{\sum_{M,N_{0}}\Omega(M,N_{0},N)e^{-\beta H(M,N_{0},N)-C}}
\end{align*}
$$

Podemos então escolher um bom valor para $C$ estudando o Hamiltoniano em jogo:
$$
\begin{align*}
H(M,N_0,N) &= \frac{N^{2}-M^{2}}{N}+D(N-N_{0})
\\[0.2cm]
&= N + DN - DN_0 - \frac{M^2}{N}
\end{align*}
$$

Escolhendo $C = -\beta(N + DN)$ temos:
$$
\begin{align*}
-\beta H(M,N_0,N) - C&= \beta\left(DN_0 + \frac{M^2}{N}\right)
\end{align*}
$$

In [3]:
# Parâmetro abaixo do qual não usamos a aproximação de Stirling
STIRLING = 10

# Memorizar o valor do fatorial para n baixo
memfactorial = [1]
for n in range(STIRLING):
    memfactorial += [memfactorial[-1] * (n+1)]

memfactorial = np.array(memfactorial)

def fastfactorial(n: int) -> int:
    """
        Calcula o fatorial dos elementos de `n`. Usa a aproximação de Stirling para inteiros maiores que o parâmetro STIRLING.
    """
    # Aceder às variáveis globais
    global STIRLING, memfactorial

    if n <= STIRLING:
        return memfactorial[n]
    else:
        return n * np.log(n) - n + np.log(2 * np.pi * n) /2

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


1. Para cada temperatura a distribuição de probabilidades da magnetização
é dada por,
\begin{equation}
P_{N,\beta}(M,N_{0})=\frac{e^{\ln(\Omega(M,N_{0},N))-\beta H(M,N_{0},N)}}{Z(\beta,N)}
\end{equation}
e a distribuição de probabilidade da magnetização é dada por, 
\begin{equation}
P_{N,\beta}(M)=\sum_{N_{0}}P_{N,\beta}(M,N_{0})
\end{equation}
Gere uma amostra de valores aleatórios de $M$ e $N_{0}$ com a distribução
pretendida usando o método von Neumann. Com esta amostra $\{M_{1},M_{2},\dots,M_{K}\}$

$\qquad$ a) Represente o histograma da amostra aleatória gerada e compare com
a distribuição teórica para $\beta\in\{0.1,0.5,0.75,0.9,1.0,1.05,1.1,1.2,1.5\}$
e $D\in\{0.0,0.25,0.5,0.75,1\}$.

Responda Aqui

In [2]:
# Responda Aqui

$\quad$(b) Meça a autocorrelação da magnetização para $\beta=1.05$,
\begin{align*}
corr_{M}(\tau)	& =\frac{\left\langle M_{i}M_{i+\tau}\right\rangle -\left\langle M_{i}\right\rangle ^{2}}{\left\langle M_{i}^{2}\right\rangle -\left\langle M_{i}\right\rangle ^{2}}\\
\left\langle M_{i}M_{i+\tau}\right\rangle &	=\frac{1}{L}\sum_{i=1}^{L}M_{i}M_{i+\tau}
\end{align*}

Responda Aqui

In [2]:
# Responda Aqui

## Pergunta 2

Na pergunta anterior efectuamos uma simulação Monte Carlo gerando as amostras da variável $M$. Será que a amostra gerada corresponde à gerada por uma dinâmica do sistema de N spins com o Hamiltoniano, 
\begin{equation*}
H=\frac{N^{2}-M^{2}}{N}+D(N-N_{0})
\end{equation*}
e com o algorítmo de Metropolis?

1. Para testar implemente o algoritmo de Metropolis para $N$ spins:

• Atribua a cada um dos $N$ spins do sistema uma variável $\pm1,0$ com probailidade uniforme.

• Considere um passo de tempo o seguinte algoritmo:

– Escolha um spin com probabilidade uniforme.

– Proponha alterar o seu valor e aceite com a probabilidade $\min\left(1,e^{-\beta\Delta H}\right)$ senão fique no mesmo estado.

Responda Aqui

In [2]:
# Responda Aqui

2. Compare as distribuições de probabilidade assiptótica da magnetização para vários valores de $\beta$ e $D$.  

Responda Aqui

In [2]:
# Responda Aqui

3. Os amostras geradas são equivalentes às do primeiro problema?

Responda Aqui

In [2]:
# Responda Aqui

### Bom Trabalho